<a href="https://colab.research.google.com/github/ShreyPandit/Knowledge_Distillation/blob/main/Knowledge_Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the required Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

Defining the the Distiller Class

In [7]:
# Defining the Distiller class that will override the keras.Model
class Distiller(keras.Model):

  # Initializing the class
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    # Overriding the compile function of the keras.Model
    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn, # Student loss function
        distillation_loss_fn, # Distillation loss function
        alpha=0.1, # How much weightage must be given to the distiller loss function
        temperature=3,
    ):  
        #Initializing the variables
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics) 
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    #Overriding the train_step function of keras.Model
    def train_step(self, data):

        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)
        
        # Here Gradient Tape is used for calculating the gradients of any variabe in the graph
        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # The Distillation Loss function 
            # Here Temperature is the constant divided so as to create a smoother prediction instead of a hard prediction
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            # Use of the Alpha as a weighted average
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [3]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [4]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))


11493376/11490434 [==============================] - 0s 0us/step


In [5]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 62s 17ms/step - loss: 0.2449 - sparse_categorical_accuracy: 0.9231
Epoch 2/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0935 - sparse_categorical_accuracy: 0.9718
Epoch 3/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0747 - sparse_categorical_accuracy: 0.9772
Epoch 4/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0711 - sparse_categorical_accuracy: 0.9781
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - loss: 0.0947 - sparse_categorical_accuracy: 0.9762


[0.09472820907831192, 0.9761999845504761]

In [8]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 14s 7ms/step - sparse_categorical_accuracy: 0.8528 - student_loss: 0.3153 - distillation_loss: 0.0941
Epoch 2/3
1875/1875 [==============================] - 13s 7ms/step - sparse_categorical_accuracy: 0.9686 - student_loss: 0.1252 - distillation_loss: 0.0297
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - sparse_categorical_accuracy: 0.9785 - student_loss: 0.0874


[0.9785000085830688, 2.5406361601199023e-06]

In [9]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4604 - sparse_categorical_accuracy: 0.8571
Epoch 2/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1019 - sparse_categorical_accuracy: 0.9695
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - loss: 0.0597 - sparse_categorical_accuracy: 0.9791


[0.05969225987792015, 0.9790999889373779]

Here we can see that the Model where knowledge was distilled from a larger model had an accuracy of 97% whereas the one trained from scratch has 59%